In [136]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [131]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

In [149]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

## XGBoost Without PCA

In [171]:
files = ['CompleteWeeklyIndexes.csv','MeanWeeklyImputed.csv', 'KNNWeeklyImputed.csv', 'MovingAverageWeeklyImputed.csv',  'RegressionWeeklyImputed.csv', 'MIIWeeklyImputed.csv']

for file in files:
    df = read_csv(file)

    print(file)
    if "MII" not in file:
        X_test = read_csv('CompleteWeeklyIndexes.csv')
    else:
        test_data = file.split('.')
        X_test = read_csv(test_data[0] + '_test.' + test_data[1])
    y = pd.read_csv('sp500_target_regimes.csv', parse_dates=True)
    y.index = y['date'].values
    y = y[['is_transition']]
       

    X_train = df.loc['2000-01-01':'2015-01-01']
    X_test = X_test.loc['2015-01-02':]
    y_train = y.loc['2000-01-01':'2015-01-01']
    y_test = y.loc['2015-01-02':]
    
    param_test1 = {
        'max_depth':range(2,6,1),
        'min_child_weight':range(1,6,1),
        #'gamma':np.arange(0.1,0.5,0.1),
        'max_delta_step':range(1,10,1),
        #'scale_pos_weight':[75, 99, 100, 1000, 10000,np.sqrt(len(y_train[y_train['is_transition']==False])/len(y_train[y_train['is_transition']==True])),len(y_train[y_train['is_transition']==False])/len(y_train[y_train['is_transition']==True])]
    }
        
    balance = np.sqrt(len(y_train[y_train['is_transition']==False])/len(y_train[y_train['is_transition']==True]))
    
    gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(objective="binary:logistic",
                                                          learning_rate=0.1, 
                                                          n_estimators=200, 
                                                          random_state=27, 
                                                          min_child_weight=2, 
                                                          gamma=2, 
                                                          max_depth=2,
                                                          max_delta_step=1,
                                                          eval_metric=['auc'], 
                                                          scale_pos_weight=balance), 
                            param_grid = param_test1, 
                            scoring='f1',
                            n_jobs=4,
                            iid=False, 
                            cv=5)

    gsearch1.fit(X_train, y_train)
    print(gsearch1.best_params_, gsearch1.best_score_)
    
    xgb_model = xgb.XGBClassifier(objective="binary:logistic", 
                                  learning_rate=0.1, 
                                  n_estimators=200, 
                                  random_state=27, 
                                  min_child_weight=gsearch1.best_params_['min_child_weight'], 
                                  max_delta_step=gsearch1.best_params_['max_delta_step'],
                                  #gamma=gsearch1.best_params_['gamma'], 
                                  max_depth=gsearch1.best_params_['max_depth'], 
                                  eval_metric=['auc'], 
                                  scale_pos_weight=balance)

    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    y_train_pred = xgb_model.predict(X_train, )
    print("_______________________________________")
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("Fscore", f_score)
    print("---------------------------------------")
    print("_______________________________________")
    y_pred = gsearch1.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("Fscore", f_score)
    print("---------------------------------------")


CompleteWeeklyIndexes.csv
{'max_delta_step': 1, 'max_depth': 2, 'min_child_weight': 1} 0.0
_______________________________________
     True False
True    5    0
False   0    777
_______________________________________
Fscore 1.0
---------------------------------------
_______________________________________
     True False
True    1    5
False   6    260
_______________________________________
Fscore 0.15384615384615383
---------------------------------------
MeanWeeklyImputed.csv
{'max_delta_step': 1, 'max_depth': 2, 'min_child_weight': 1} 0.0
_______________________________________
     True False
True    5    0
False   0    777
_______________________________________
Fscore 1.0
---------------------------------------
_______________________________________
     True False
True    1    3
False   6    262
_______________________________________
Fscore 0.18181818181818182
---------------------------------------
KNNWeeklyImputed.csv
{'max_delta_step': 1, 'max_depth': 2, 'min_child_weig

## XGBoost With PCA

In [172]:
files = ['MeanWeeklyImputedPCA.csv', 'KNNWeeklyImputedPCA.csv', 'MovingAverageWeeklyImputedPCA.csv', 'RegressionWeeklyImputedPCA.csv', 'MIIWeeklyImputedPCA.csv']
param_test1 = {
    'max_depth':range(2,10,1),
    'min_child_weight':range(1,6,2),
    'gamma':range(1,6,1)
}
for file in files:
    df = read_csv(file)
    if 'Weekly' in file:
        test_data = file.split('.')
        print(file, test_data[0] + '_test.' + test_data[1])
        X_test = read_csv(test_data[0] + '_test.' + test_data[1])
        y = pd.read_csv('sp500_target_regimes.csv', parse_dates=True)
        y.index = y['date'].values
        y = y[['is_transition']]
       
    else:
        continue
        #X_test = read_csv('CompleteIndexes.csv')
        #X_test = X_test.loc['2015-01-02':]
    
    X_train = df 
    y_train = y.loc['2000-01-01':'2015-01-01']
    y_test = y.loc['2015-01-02':]
    
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    balance = len(y_train[y_train['is_transition']==False])/len(y_train[y_train['is_transition']==True])
    
    gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(objective="binary:logistic", 
                                                          learning_rate=0.1,
                                                          n_estimators=200, 
                                                          random_state=27, 
                                                          min_child_weight=2, 
                                                          gamma=3, 
                                                          max_depth=2, 
                                                          eval_metric=['auc'], 
                                                          scale_pos_weight=balance), 
                            param_grid = param_test1, 
                            scoring='f1',
                            n_jobs=4,
                            iid=False, 
                            cv=5)

    gsearch1.fit(X_train, y_train, eval_metric=F1_eval)
    print(gsearch1.best_params_, gsearch1.best_score_)
    
    xgb_model = xgb.XGBClassifier(objective="binary:logistic", 
                                  learning_rate=0.1, 
                                  n_estimators=500, 
                                  random_state=27,
                                  min_child_weight=gsearch1.best_params_['min_child_weight'],
                                  gamma=gsearch1.best_params_['gamma'], 
                                  max_depth=gsearch1.best_params_['max_depth'], 
                                  eval_metric=['auc'], 
                                  scale_pos_weight=balance)

    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, eval_metric=F1_eval)
    
    y_train_pred = xgb_model.predict(X_train, )
    print("_______________________________________")
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("Fscore", f_score)
    print("---------------------------------------")
    print("_______________________________________")
    y_pred = gsearch1.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("Fscore", f_score)
    print("---------------------------------------")


MeanWeeklyImputedPCA.csv MeanWeeklyImputedPCA_test.csv
(782, 14) (782, 1) (272, 14) (272, 1)
{'gamma': 1, 'max_depth': 2, 'min_child_weight': 1} 0.0
_______________________________________
     True False
True    5    0
False   0    777
_______________________________________
Fscore 1.0
---------------------------------------
_______________________________________
     True False
True    0    0
False   7    265
_______________________________________
Fscore nan
---------------------------------------
KNNWeeklyImputedPCA.csv KNNWeeklyImputedPCA_test.csv
(782, 12) (782, 1) (272, 12) (272, 1)
{'gamma': 1, 'max_depth': 2, 'min_child_weight': 1} 0.0
_______________________________________
     True False
True    5    0
False   0    777
_______________________________________
Fscore 1.0
---------------------------------------
_______________________________________
     True False
True    0    0
False   7    265
_______________________________________
Fscore nan
----------------------------